# Loading model from MLflow for inference ⬇️

By now, we have trained a model (or models) and wrote it to MLflow. 

Since our MLflow server is accessible over the internet, we can now load our model from anywhere. 🌐 <br>
This is particularly useful when we want to deploy an application later on, which relies on our model. <br>
But for now, let's look at how we can load our model back into this notebook.

In [6]:
import mlflow
import pandas as pd
import plotly.express as px

Again we first need to connect to the server:

In [7]:
mlflow.set_tracking_uri("http://20.4.198.104:5000")

Let's load our production model:

In [8]:
stage = "production"
model_name = "turbine-model"  # or choose the name you have given your model in the previous notebook 

model = mlflow.sklearn.load_model(f"models:/{model_name}/{stage}")

And apply it to some data!

But... which features did this model use? <br>
We need to know these to pass the right data through the model. <br>
Let's write use a function to load the model's features used.

In [9]:
def get_feature_names(model_name: str, stage: str):
    """Get the feature names from the model metadata, given its name and stage."""
    
    client = mlflow.tracking.MlflowClient()
    model_version = client.get_latest_versions(name=model_name, stages=[stage])[0]
    run_id = model_version.run_id
    run = client.get_run(run_id)
    features = run.data.params["features"]
    return features.split(", ")

features = get_feature_names(model_name, stage)
print(f"Features used by '{model_name}' in '{stage}':\n {features}")

Features used by 'turbine-model' in 'production':
 ['wind_speed']


Alright, now apply it to some data! 

In [10]:
data_path = "../data/turbine-data.csv"
data = pd.read_csv(data_path).set_index("timestamp")
data.index = pd.to_datetime(data.index)

In [11]:
data_without_na = data.dropna()

X = data_without_na[features]  # Exercise: what goes in the brackets?  
y = data_without_na["active_power"]

data.loc[X.index, "predictions"] = model.predict(X)

# Evaluate
score = model.score(X, y)
score

0.7903349108676633

In [12]:
plot_start = pd.Timestamp("2018-12-15")
fig = px.line(
    data[plot_start:],
    y=["active_power", "predictions"]
)
fig

And why not log it back to MLflow?

In [ ]:
mlflow.log_metric("R2_score", score)
mlflow.log_figure(fig, "predictions.html")

# Can you see it in the UI?

## Packaging our solutions 📦

To summarize: we now have trained a model locally, logged it to MLflow, and loaded it again for inference. 🎉 <br>
So far, however, all of this was done in a notebook. Unfortunaly, many ML projects with great potential get stuck in this phase.

The question now is:
- How do we turn our so-far notebook solution into a production-ready *application*? One that other people could use?

In terms of usability, we are doing quite well already, as we have logged our model to MLflow. <br>
However, we cannot expect everyone to know how to open a notebook, load our model, and get the predictions they want.

From now on, we will therefore structure our code in python files, inside a package that others can *install*.
Furthermore, we will create a RestAPI that allows others to send data to our model and get predictions back.

Have a look at the `/turbine_power` folder, and inspect the inference code, which includes the same logic as above.

In [ ]:
# Since we have already installed our own package during setup, we can also run it here:

from turbine_power.inference import run_inference

run_inference()